In [425]:
using Distributed, Folds, FLoops
using Random, Distributions

In [434]:
include("perm_test.jl")
include("partition.jl")

partition (generic function with 1 method)

In [418]:
n, n1, n2 = 200, 12, 8
d = Normal()
delta_true = 0
parts = partition(n1, n2)
Random.seed!(123)
x1s = rand(d, (n, n1))
x2s = rand(d, (n, n2))
data = hcat(x1s, x2s)

200×20 Matrix{Float64}:
 -0.645731   -0.605691   -0.756464   …   0.037911   -0.609532    0.268983
 -1.46325    -0.354813    0.21732       -1.05255    -0.236771   -1.57179
 -1.6236     -0.782707   -0.737335      -0.137629    0.33269    -0.171054
 -0.217665    0.0812944  -1.98671        0.0656706   0.526011   -0.562248
  0.492246   -0.453337    0.572599       0.406255    0.185543    0.251055
  0.98098     0.0983998  -0.979219   …  -0.736835    0.135897   -0.22668
  0.0799568   0.77467     1.72272       -0.561219   -0.357161   -0.65838
  1.54912    -2.08215    -1.37706        0.467577   -0.782272    0.304469
 -1.34161    -0.0387583  -1.3131         0.0614698   0.506474    2.50715
  0.412162   -0.701248   -0.0490801     -2.12121    -1.38048    -1.56417
  0.593197    1.30727    -0.521194   …   0.543751   -1.26496     0.494784
 -0.768409   -0.458261   -0.330495       0.842038   -0.120105    1.00935
 -0.0761679   0.232625   -0.319317      -0.52795    -0.583295    0.0196977
  ⋮                

In [421]:
first = data[1,:]
@time permInterval(first, parts, delta_true)

  0.181694 seconds (1.40 k allocations: 492.455 MiB, 2.89% gc time)


true

In [455]:
@time @distributed (+) for i in 1:n
    permInterval(data[i, 1:n1], data[i, n1+1:end], parts, delta_true)
end

 33.557403 seconds (317.95 k allocations: 96.305 GiB, 6.57% gc time, 0.16% compilation time)


194

In [442]:
@time @floop for row in eachrow(data)
    @reduce(coverage += permInterval(row[1:n1], row[n1+1:end], parts, delta_true))
end

 24.784075 seconds (819.67 k allocations: 96.331 GiB, 18.45% gc time, 2.78% compilation time)


In [440]:
@time Folds.reduce(+, Folds.map(x -> permInterval(x[1:n1], x[n1+1:end], parts, delta_true), eachrow(data)))

 23.808151 seconds (772.82 k allocations: 96.329 GiB, 18.25% gc time, 1.62% compilation time)


194